# 더미 데이터 만들기
- 언슬로스 잘쓸려면 대충 1000개 정도 더미 만드는게 필요할듯
- 이제 나온 나온 모델 있으니까 그걸 통해서 돌려보기
- 구조가 이제 나름 확정적이라 정확하게 돌리는게 중요함

- 만드는 순서
1. 주제 늘리기(100개 -> 1000개)
2. 아이디어 입력 부분 늘리기

In [1]:
import pandas as pd
import openai
import os
from dotenv import load_dotenv
import json
import slack_alarm

load_dotenv()
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
# 유저 개요 부분 출력 함수
# 최적화된 시스템 프롬프트
OPTIMIZED_SYSTEM_PROMPT = """
당신은 실제 개발자나 학생이 프로젝트 계획서나 포트폴리오에 작성할 법한 자연스럽고 상세한 프로젝트 개요를 생성하는 전문 작성자입니다.

## 작성 스타일과 톤:
- 개발자나 학생이 직접 작성한 것처럼 자연스럽고 진정성 있는 문체
- 기술적 내용과 목표를 명확하게 설명하되, 과도하게 마케팅적이지 않은 실용적 접근
- 프로젝트의 배경, 목적, 주요 기능, 기술 스택, 기대 효과를 논리적 순서로 설명
- 실제 구현 과정에서의 고려사항이나 도전 과제도 자연스럽게 포함

## 개요 구성 요소:
1. **프로젝트 소개**: 서비스/시스템이 무엇인지, 어떤 문제를 해결하는지
2. **핵심 목표**: 프로젝트를 통해 달성하고자 하는 구체적 목표
3. **주요 기능**: 사용자가 실제로 사용할 수 있는 기능들을 구체적으로 나열
4. **기술적 구현**: 사용된 기술 스택과 아키텍처 선택의 이유
5. **기대 효과**: 사용자에게 제공하는 가치와 프로젝트의 의의

## 작성 원칙:
- 문장이 자연스럽게 이어지며 읽기 쉬운 단락 구성
- 기술 용어를 적절히 사용하되 과도하지 않게
- 실제 개발 과정에서 고려할 법한 세부사항 포함
- 프로젝트의 실용성과 완성도를 강조
- 200-400자 내외의 적절한 길이로 상세하면서도 간결하게

## 피해야 할 요소:
- 과도하게 홍보성인 문구나 과장된 표현
- 추상적이거나 모호한 설명
- 기술 나열만 하는 건조한 문체
- 마케팅 문구 같은 부자연스러운 표현

**중요: JSON이나 구조화된 형식이 아닌, 하나의 연속된 문단으로 자연스럽게 작성하세요.**
"""

MODEL_ID = "gpt-4o"

def generate_project_subject() :   
  # 간소화된 프롬프트
  enhanced_prompt = f"""
  프로젝트 개요를 하나의 자연스러운 문단으로 작성해주세요.

  포함 요소:
  - 프로젝트가 무엇인지
  - 어떤 문제를 해결하는지  
  - 주요 기능 3-4개
  - 사용 기술 스택
  - 기대 효과

  예시 스타일: "○○ 서비스는 △△ 문제를 해결하기 위한 ××로, ~을 목표로 한다. 사용자들은 ~할 수 있으며, ~기능을 통해 ~를 제공한다. 서비스는 ~기술을 기반으로 구현되었으며..."

  텍스트만 응답하세요.
  """
  try:
      response = client.chat.completions.create(
          model=MODEL_ID,
          messages=[
              {"role": "system", "content": OPTIMIZED_SYSTEM_PROMPT},
              {"role": "user", "content": enhanced_prompt}
          ],
          max_tokens=3000,
          temperature=1.3
      )
      generated_text = response.choices[0].message.content
      return generated_text

  except openai.APIError as e:
      print(f"OpenAI API 오류: {e}")
      return None
  except Exception as e:
      print(f"예상치 못한 오류: {e}")
      return None

In [ ]:
# 개요 부분 출력
data_lst = []  

for i in range (0, 900) :
  generate_data = generate_project_subject()
  data_lst.append(generate_data)

df = pd.read_csv("subject.csv")
df_lst = df['text'].tolist()
total_lst = df_lst + data_lst
total_df = pd.DataFrame(columns = ['subject'], data = total_lst)
total_df.to_csv('1.subject.csv', index = False)
slack_alarm.send_notification("출력 완료")

True

# 2번 유저 데이터 생성

In [ ]:
# 유저 데이터 더미 생성 함수

client = openai.OpenAI()
# 최적화된 시스템 프롬프트
OPTIMIZED_SYSTEM_PROMPT = """
당신은 소프트웨어 프로젝트 더미 데이터를 생성하는 전문가입니다.

## 역할과 목표
- 실제 개발 프로젝트에서 사용할 수 있는 현실적이고 구체적인 더미 데이터를 생성합니다
- 다양한 분야(AI, 웹, 모바일, 데이터분석, IoT 등)의 프로젝트를 균형있게 생성합니다
- 최신 기술 트렌드를 반영한 기술 스택을 제안합니다

## 생성 원칙
1. **현실성**: 실제로 구현 가능한 프로젝트여야 합니다
2. **구체성**: 모호하지 않고 명확한 기능과 목표를 가져야 합니다  
3. **다양성**: 서로 다른 분야와 기술 스택을 사용해야 합니다
4. **일관성**: 프로젝트명, 대상, 기능, 기술스택이 서로 논리적으로 연결되어야 합니다
5. **최신성**: 2023-2024년 기준 최신 기술과 트렌드를 반영해야 합니다

## 기술 스택 가이드라인
- **AI/ML**: GPT-4, Claude, Llama, TensorFlow, PyTorch, OpenCV
- **백엔드**: Python, Node.js, Java, Go, Django, FastAPI, Express.js
- **프론트엔드**: React, Vue.js, Angular, Next.js, TypeScript
- **모바일**: React Native, Flutter, Swift, Kotlin
- **데이터베이스**: PostgreSQL, MongoDB, Redis, MySQL, Elasticsearch
- **클라우드**: AWS, Google Cloud, Azure, Docker, Kubernetes

## 응답 형식
- 반드시 JSON 배열 형태로만 응답하세요
- 추가 설명이나 주석 없이 순수한 JSON만 제공하세요
- 모든 텍스트는 한국어로 작성하세요
- 각 필드는 요청된 형식을 정확히 따라야 합니다"""

# 파인튜닝된 모델 ID
MODEL_ID = 'gpt-4o-mini'

def user_dummy_create(project_overview):
  # 구조화된 요청 프롬프트 (f-string 사용하지 않음)
    enhanced_prompt = f"""
    프로그램 주제 :
    {project_overview}
    
    다음 조건에 맞는 프로젝트 더미 데이터를 JSON 형식으로 아래와 같이 생성해주세요

    ## 생성 조건
    - **프로젝트 이름**: 창의적이고 실제적인 프로젝트명
    - **프로젝트 대상**: 구체적인 타겟 사용자나 업계
    - **메인 기능**: 각 프로젝트마다 3개의 핵심 기능 (AI, 웹, 모바일, 데이터 분석 등 다양한 분야 포함)
    - **기술 스택**: 각 프로젝트마다 3개의 기술 스택 (프로그래밍 언어, 프레임워크, 데이터베이스, AI 모델 등)
    - **프로젝트 설명**: 프로그램 주제와 동일하게 작성해주세요

    ## JSON 형식
    {{
      "projectName": "프로젝트 이름",
      "projectTarget": "프로젝트 대상",
      "mainFunction": [
        "메인 기능1",
        "메인 기능2", 
        "메인 기능3"
      ],
      "techStack": [
        "기술 스택1",
        "기술 스택2",
        "기술 스택3"
      ],
      "projectDescription": "프로젝트 설명"
    }}

    **요청사항**: 
    1. 서로 다른 분야의 프로젝트 **1개만** 생성해주세요
    2. 각 프로젝트는 현실적이고 구현 가능한 수준이어야 합니다
    3. 응답은 JSON 배열 형태로만 제공해주세요 (다른 설명 없이)
    4. 한국어로 작성해주세요

    예시:
    [
      {{
        "projectName": "SmartERD Generator",
        "projectTarget": "소프트웨어 개발자 및 시스템 설계자",
        "mainFunction": [
          "AI 기반 프로젝트 ERD 자동 생성",
          "대시보드에서 실시간 로그 모니터링",
        ],
        "techStack": [
          "Llama 3.1",
          "Python 3.8",
        ],
        "projectDescription": "{project_overview}."
      }}
    ]
    """
    try:
        response = client.chat.completions.create(
            model=MODEL_ID,
            messages=[
                {"role": "system", "content": OPTIMIZED_SYSTEM_PROMPT},
                {"role": "user", "content": enhanced_prompt}
            ],
            max_tokens=4000,
            temperature=0.3
        )
        
        return response.choices[0].message.content
        
    except Exception as e:
        return f"오류 발생: {e}"

In [ ]:
data_lst = []
i = 1
for project_overview in total_lst :
  user_input = user_dummy_create(project_overview)
  data_lst.append(user_input)
  print(f'전체 {len(total_lst)} 중 {i}번째 진행중')
  if i % 50 == 0 :
    slack_alarm.send_notification(f"{i}번째 작성 진행중")
  i += 1

total_df = pd.DataFrame(columns = ['user_input'], data = data_lst)
total_df.to_csv('2.user_input.csv', index = False)
slack_alarm.send_notification("2번 유저 프로젝트 입력 출력")

전체 1000 중 1번째 진행중
전체 1000 중 2번째 진행중
전체 1000 중 3번째 진행중
전체 1000 중 4번째 진행중
전체 1000 중 5번째 진행중
전체 1000 중 6번째 진행중
전체 1000 중 7번째 진행중
전체 1000 중 8번째 진행중
전체 1000 중 9번째 진행중
전체 1000 중 10번째 진행중
전체 1000 중 11번째 진행중
전체 1000 중 12번째 진행중
전체 1000 중 13번째 진행중
전체 1000 중 14번째 진행중
전체 1000 중 15번째 진행중
전체 1000 중 16번째 진행중
전체 1000 중 17번째 진행중
전체 1000 중 18번째 진행중
전체 1000 중 19번째 진행중
전체 1000 중 20번째 진행중
전체 1000 중 21번째 진행중
전체 1000 중 22번째 진행중
전체 1000 중 23번째 진행중
전체 1000 중 24번째 진행중
전체 1000 중 25번째 진행중
전체 1000 중 26번째 진행중
전체 1000 중 27번째 진행중
전체 1000 중 28번째 진행중
전체 1000 중 29번째 진행중
전체 1000 중 30번째 진행중
전체 1000 중 31번째 진행중
전체 1000 중 32번째 진행중
전체 1000 중 33번째 진행중
전체 1000 중 34번째 진행중
전체 1000 중 35번째 진행중
전체 1000 중 36번째 진행중
전체 1000 중 37번째 진행중
전체 1000 중 38번째 진행중
전체 1000 중 39번째 진행중
전체 1000 중 40번째 진행중
전체 1000 중 41번째 진행중
전체 1000 중 42번째 진행중
전체 1000 중 43번째 진행중
전체 1000 중 44번째 진행중
전체 1000 중 45번째 진행중
전체 1000 중 46번째 진행중
전체 1000 중 47번째 진행중
전체 1000 중 48번째 진행중
전체 1000 중 49번째 진행중
전체 1000 중 50번째 진행중
전체 1000 중 51번째 진행중
전체 1000 중 52번째 진행중
전체 1000 중 53번째 진행중
전체

True

In [ ]:
# json.loads로 받을 생각 안하고 진행했는데 오류 났음
## 아마 개요 뽑을때 Temperature = 1.3으로 해서 개요 뽑을때 오류난 느낌인거 같은데
### 테스트 셋에선 이걸 제외하고 진행하겠음

import numpy as np
import pandas as pd

data_json_lst = []
for i in data_lst:
    try:
        data_json_lst.append(json.loads(i))
    except:
        print(len(data_json_lst))
        data_json_lst.append(None)

json_df = pd.DataFrame({'user_input': data_json_lst})
json_df.to_csv('2.user_input.csv', index=False)
slack_alarm.send_notification("2번 유저 프로젝트 입력 출력")

111
130
462
471
831
914
975


True

# 데이터 결측값 정리

In [96]:
subject_df = pd.read_csv("1.subject.csv")
user_df = pd.read_csv("2.user_input.csv")

merge_df = subject_df.copy()
merge_df['user_input'] = user_df['user_input'].tolist()
merge_drop = merge_df.dropna()
merge_drop.head(5)

,subject,user_input
0,"GQ 서비스는 학습 중심의 스터디 그룹 운영을 지원하는 웹 플랫폼으로, 단순한 그룹...","[{'projectName': 'StudyBuddy', 'projectTarget'..."
1,‘기억박물관’은 치매 환자와 그 가족을 위한 따뜻한 기억 회복 웹사이트입니다. 사용...,"[{'projectName': '기억의 다리', 'projectTarget': '치..."
2,"이 앱은 보행 약자들을 위한 지하철 길찾기 앱으로, 계단 없이 이동할 수 있는 경로...","[{'projectName': 'EasyTransit Navigator', 'pro..."
3,"이 앱은 1인 가구를 위한 맞춤형 플랫폼으로, 주로 커뮤니티 기능에 중점을 두고 있...","[{'projectName': '이웃과의 소통 플랫폼', 'projectTarget..."
4,"이 웹앱은 응급실 및 병원 찾기 기능을 제공하여, 응급 상황에서 사용자가 빠르고 안...","[{'projectName': 'EmergencyCare Finder', 'proj..."


# 요구사항 명세서 초기 단계 작성(최소 3개)

In [43]:
client = openai.OpenAI()

# 최적화된 시스템 프롬프트
OPTIMIZED_SYSTEM_PROMPT = """
당신은 소프트웨어 프로젝트의 요구사항 분석 전문가입니다.
주어진 프로젝트 개요를 바탕으로 현실적이고 구체적인 기능 요구사항과 성능 요구사항을 생성하는 것이 목표입니다.

생성 규칙:
1. 기능 요구사항(FUNCTIONAL): 사용자 관점에서 시스템이 제공해야 하는 구체적인 기능
2. 성능 요구사항(PERFORMANCE): 시스템의 응답시간, 처리량, 가용성, 확장성 등 측정 가능한 수치 포함
3. 각 요구사항은 명확하고 구현 가능하며 테스트 가능해야 함
4. 프로젝트 규모와 성격에 적합한 현실적인 수준이어야 함
5. 반드시 지정된 JSON 형식으로만 응답할 것
"""

# 파인튜닝된 모델 ID
MODEL_ID = 'gpt-4o-mini'

def requirements_dummy_create(project_overview, user_input):
  # 구조화된 요청 프롬프트 (f-string 사용하지 않음)
    enhanced_prompt = f"""
    프로젝트 개요: {project_overview}
    유저가 작성한 프로젝트 정보 : {user_input}

    위 프로젝트에 대해 다음 조건에 맞는 요구사항을 생성해주세요:

    **생성 조건:**
    - 기능 요구사항(FUNCTIONAL) 3개
    - 성능 요구사항(PERFORMANCE) 3개
    - 총 6개의 요구사항을 JSON 배열 형태로 생성

    **중요: 반드시 아래 JSON 형식으로만 응답하세요. 다른 텍스트, 설명, 주석은 절대 포함하지 마세요.**

    JSON 형식:
    [
      {{"requirementType": "FUNCTIONAL1", "content": "구체적인 기능 요구사항"}},
      {{"requirementType": "FUNCTIONAL2", "content": "구체적인 기능 요구사항"}},
      {{"requirementType": "FUNCTIONAL3", "content": "구체적인 기능 요구사항"}},
      {{"requirementType": "PERFORMANCE1", "content": "구체적인 성능 요구사항 (수치 포함)"}},
      {{"requirementType": "PERFORMANCE2", "content": "구체적인 성능 요구사항 (수치 포함)"}},
      {{"requirementType": "PERFORMANCE3", "content": "구체적인 성능 요구사항 (수치 포함)"}}
    ]

    응답은 반드시 위 JSON 배열로만 시작하고 끝나야 합니다.
    """
    try:
        response = client.chat.completions.create(
            model=MODEL_ID,
            messages=[
                {"role": "system", "content": OPTIMIZED_SYSTEM_PROMPT},
                {"role": "user", "content": enhanced_prompt}
            ],
            max_tokens=4000,
            temperature=0.3
        )
        
        return response.choices[0].message.content
        
    except Exception as e:
        return f"오류 발생: {e}"

In [97]:
requirements_lst = []
i = 1
for project_overview, user_input in zip(merge_drop['subject'], merge_drop['user_input']) :

  try:
    requirement = json.loads(requirements_dummy_create(project_overview, user_input))
    requirements_lst.append(requirement)
    print(f'전체 {len(merge_drop)} 중 {i}번째 진행중')
  except (TypeError, json.JSONDecodeError, Exception) as e :
    requirements_lst.append(None)
    print(f'전체 {len(merge_drop)} 중 {i}번째 오류 발생: {type(e).__name__}')

  if i % 50 == 0 :
    slack_alarm.send_notification(f"{i}번째 작성 진행중")
  i += 1

merge_drop['requirements'] = requirements_lst
merge_drop.to_csv('3.requirements.csv', index = False)
slack_alarm.send_notification("3번 요구사항 명세서 작업 완료")

전체 993 중 1번째 진행중
전체 993 중 2번째 진행중
전체 993 중 3번째 진행중
전체 993 중 4번째 진행중
전체 993 중 5번째 진행중
전체 993 중 6번째 진행중
전체 993 중 7번째 진행중
전체 993 중 8번째 진행중
전체 993 중 9번째 진행중
전체 993 중 10번째 진행중
전체 993 중 11번째 진행중
전체 993 중 12번째 진행중
전체 993 중 13번째 진행중
전체 993 중 14번째 진행중
전체 993 중 15번째 진행중
전체 993 중 16번째 진행중
전체 993 중 17번째 진행중
전체 993 중 18번째 진행중
전체 993 중 19번째 진행중
전체 993 중 20번째 진행중
전체 993 중 21번째 진행중
전체 993 중 22번째 진행중
전체 993 중 23번째 진행중
전체 993 중 24번째 진행중
전체 993 중 25번째 진행중
전체 993 중 26번째 진행중
전체 993 중 27번째 진행중
전체 993 중 28번째 진행중
전체 993 중 29번째 진행중
전체 993 중 30번째 진행중
전체 993 중 31번째 진행중
전체 993 중 32번째 진행중
전체 993 중 33번째 진행중
전체 993 중 34번째 진행중
전체 993 중 35번째 진행중
전체 993 중 36번째 진행중
전체 993 중 37번째 진행중
전체 993 중 38번째 진행중
전체 993 중 39번째 진행중
전체 993 중 40번째 진행중
전체 993 중 41번째 진행중
전체 993 중 42번째 진행중
전체 993 중 43번째 진행중
전체 993 중 44번째 진행중
전체 993 중 45번째 진행중
전체 993 중 46번째 진행중
전체 993 중 47번째 진행중
전체 993 중 48번째 진행중
전체 993 중 49번째 진행중
전체 993 중 50번째 진행중
전체 993 중 51번째 진행중
전체 993 중 52번째 진행중
전체 993 중 53번째 진행중
전체 993 중 54번째 진행중
전체 993 중 55번째 진행중
전체 993 중 56번째 진행중
전

C:\Users\mir96\AppData\Local\Temp\ipykernel_23176\4182052518.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge_drop['requirements'] = requirements_lst


True